In [1]:
from datetime import datetime, timedelta
import pickle
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
import re

from tldextract import extract
import langid

pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_rows', 50)

from newspaper import Article

<ipython-input-1-6e87932399ef>:12: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


NOTE: Mediacloud news for state-specific collection includes news ABOUT the particular state, and as such, might sometimes contain news from national outlets where the article itself might be focused on the state itself. Most (if not all) of the collection is going to be sourced from local state-specific outlets, but not always. Currently, we are keeping all URLs within the obtained collection for the state, except those from nytimes and foxnews since those are our chosen national outlets. Trusting mediacloud's collections to be truly state-focused even when urls come from national outlets. 

In [2]:
start_date = datetime.strptime('2023-04-01', '%Y-%m-%d').date()
end_date = datetime.strptime('2023-06-30', '%Y-%m-%d').date()

In [3]:
texas_news = pd.read_csv('data/mediacloud_texas.csv')
print(texas_news.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4014 entries, 0 to 4013
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   ap_syndicated         4014 non-null   bool   
 1   collect_date          4014 non-null   object 
 2   feeds                 0 non-null      float64
 3   guid                  4014 non-null   object 
 4   language              4014 non-null   object 
 5   media_id              4014 non-null   int64  
 6   media_name            4014 non-null   object 
 7   media_url             4014 non-null   object 
 8   metadata              4014 non-null   object 
 9   processed_stories_id  4014 non-null   int64  
 10  publish_date          4014 non-null   object 
 11  stories_id            4014 non-null   int64  
 12  story_tags            4014 non-null   object 
 13  title                 4014 non-null   object 
 14  url                   4014 non-null   object 
 15  word_count           

In [4]:
texas_news[['language', 'media_name', 'media_url', 'publish_date', 'title', 'url']].tail(20)

,language,media_name,media_url,publish_date,title,url
3994,en,ksat.com,http://ksat.com/,2023-06-30 16:00:16,Supreme Court lets stand ruling that protects people with gender dysphoria under disability law,https://www.ksat.com/news/national/2023/06/30/supreme-court-lets-stand-ruling-that-protects-people-with-gender-dysphoria-under-disability-law/
3995,en,cbs19.tv,http://cbs19.tv/,2023-06-30 11:12:17,The Supreme Court rules for a designer who doesn't want to make wedding websites for gay couples,https://feeds.feedblitz.com/~/749807159/0/cbs19/news~The-Supreme-Court-rules-for-a-designer-who-doesnt-want-to-make-wedding-websites-for-gay-couples
3996,en,cbs19.tv,http://cbs19.tv/,2023-06-30 09:17:50,Transgender influencer Dylan Mulvaney speaks out after Bud Light backlash,https://feeds.feedblitz.com/~/749870372/0/cbs19/news~Transgender-influencer-Dylan-Mulvaney-speaks-out-after-Bud-Light-backlash
3997,en,texaslawyer.com,http://texaslawyer.com/,2023-06-30 16:02:24,'Frivolous Litigation Is Not the Way to Resolve Disputes or Effect Change': Sorority Files Motion to Dismiss Members' Lawsuit Over Trans Member,https://feeds.feedblitz.com/-/749870309/0/law/legal-news/
3998,en,Breitbart - United States - Texas,http://www.breitbart.com/texas/,2023-06-30 15:41:00,Proposed Michigan Hate Speech Law Would Criminalize Making Someone 'Feel' Frightened or Intimidated,https://www.breitbart.com/politics/2023/06/30/proposed-michigan-hate-speech-law-would-criminalize-making-someone-feel-frightened-or-intimidated/
3999,en,Breitbart - United States - Texas,http://www.breitbart.com/texas/,2023-06-30 15:42:17,"Disney Stock Downgraded Amid Concerns Streaming Growth Has Stalled, Weak Theme Park Attendance",https://www.breitbart.com/entertainment/2023/06/30/disney-stock-downgraded-amid-concerns-streaming-growth-has-stalled-weak-theme-park-attendance/
4000,en,click2houston.com,http://www.click2houston.com#spider,2023-06-30 16:00:16,Supreme Court lets stand ruling that protects people with gender dysphoria under disability law,https://www.click2houston.com/news/national/2023/06/30/supreme-court-lets-stand-ruling-that-protects-people-with-gender-dysphoria-under-disability-law/
4001,en,click2houston.com,http://www.click2houston.com#spider,2023-06-30 10:04:46,The Supreme Court rules for a designer who doesn't want to make wedding websites for gay couples,https://www.click2houston.com/business/2023/06/30/the-supreme-court-rules-for-a-designer-who-doesnt-want-to-make-wedding-websites-for-gay-couples/
4002,en,WBAP,http://wbap.com/,2023-06-30 10:19:19,Supreme Court Rules in Favor of Christian Web Designer Over LGBTQ Designs,https://www.wbap.com/2023/06/30/supreme-court-rules-in-favor-of-christian-web-designer-over-lgbtq-designs/
4003,en,dfw.cbslocal.com,http://dfw.cbslocal.com/,2023-06-30 18:46:33,"LGBTQ Texas lawmakers 'scared', 'tired' of political targeting of their community",https://www.cbsnews.com/texas/news/lgbtq-texas-lawmakers-scared-tired-of-political-targeting-of-their-community/


In [12]:
relevant_news = texas_news[~texas_news['media_name'].isin(['Fox News', 'New York Times'])]
relevant_news = relevant_news[relevant_news['language']=='en']
relevant_news = relevant_news[['media_name', 'publish_date', 'title', 'url']]
print(relevant_news.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3923 entries, 0 to 4013
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   media_name    3923 non-null   object
 1   publish_date  3923 non-null   object
 2   title         3923 non-null   object
 3   url           3923 non-null   object
dtypes: object(4)
memory usage: 153.2+ KB
None


In [13]:
relevant_news.publish_date = pd.to_datetime(relevant_news.publish_date)
relevant_news = relevant_news[relevant_news['publish_date'].dt.date >= start_date]
relevant_news = relevant_news[relevant_news['publish_date'].dt.date <= end_date]
print(relevant_news.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3923 entries, 0 to 4013
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   media_name    3923 non-null   object        
 1   publish_date  3923 non-null   datetime64[ns]
 2   title         3923 non-null   object        
 3   url           3923 non-null   object        
dtypes: datetime64[ns](1), object(3)
memory usage: 153.2+ KB
None


In [14]:
def get_article_from_url(url):
    try:
        article = Article(url)
        article.download()
        article.parse()
        subtitle = article.meta_description
        if subtitle == '' or subtitle == ' ':
            subtitle = None
        text = article.text
        if text == '' or text == ' ':
            text = None
    except:
        subtitle = None
        text = None
    return subtitle, text

In [15]:
subtitles, texts = [], []
all_urls = list(relevant_news['url'])
for url in tqdm(all_urls):
    subtitle, text = get_article_from_url(url)
    subtitles.append(subtitle)
    texts.append(text)
relevant_news['subtitle'] = subtitles
relevant_news['text'] = texts
print(relevant_news.info())

  0%|          | 1/3923 [00:00<27:09,  2.41it/s]/home/pranavgoel/miniconda3/envs/pg3/lib/python3.8/site-packages/dateutil/parser/_parser.py:1207: UnknownTimezoneWarning: tzname CDT identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "
 67%|██████▋   | 2637/3923 [28:09<06:29,  3.30it/s]  /home/pranavgoel/miniconda3/envs/pg3/lib/python3.8/site-packages/PIL/TiffImagePlugin.py:781: UserWarning: Possibly corrupt EXIF data.  Expecting to read 2060 bytes but only got 719. Skipping tag 59932
  warnings.warn(
/home/pranavgoel/miniconda3/envs/pg3/lib/python3.8/site-packages/PIL/TiffImagePlugin.py:802: UserWarning: Corrupt EXIF data.  Expecting to read 2 bytes but only got 0. 
  warnings.warn(str(msg))
/home/pranavgoel/miniconda3/envs/pg3/lib/python3.8/site-packages/PIL/TiffImagePlugin.py:781: UserWarning: Possibly corrupt

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3923 entries, 0 to 4013
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   media_name    3923 non-null   object        
 1   publish_date  3923 non-null   datetime64[ns]
 2   title         3923 non-null   object        
 3   url           3923 non-null   object        
 4   subtitle      2593 non-null   object        
 5   text          2628 non-null   object        
dtypes: datetime64[ns](1), object(5)
memory usage: 214.5+ KB
None


In [16]:
relevant_news.to_csv('data/texas_article_texts_and_info.csv',
                     index=False)